# Part 2: Mining the web: Web APIs

We hope the preceding exercise was painful: even with tools to process HTML, it is rough downloading raw HTML and trying to extract information from it!

> Can you think of any other reasons why scraping websites for data in this way is not a good idea?

Luckily, many websites provide an application programming interface (API) for querying their data or otherwise accessing their services from your programs. For instance, Twitter provides a web API for gathering tweets, Flickr provides one for gathering image data, and Github for accessing information about repository histories.

These kinds of web APIs are much easier to use than, for instance, the preceding technique which scrapes raw web pages and then has to parse the resulting HTML. Moreover, there are more scalable in the sense that the web servers can transmit structured data in a less verbose form than raw HTML.

As a starting example, here is some code to look at the activity on Github related to the public version of one of our course's materials.

In [1]:
from IPython.display import display, Markdown

import requests
response = requests.get('https://api.github.com/repos/hpcgarage/prymer/events')

headers_highlighted = str(response.headers).replace("'Content-Type': 'application/json;",
                                                    "'Content-Type': '**application/json**;")
display(Markdown(headers_highlighted))

{'Server': 'GitHub.com', 'Date': 'Sun, 03 Oct 2021 06:20:45 GMT', 'Content-Type': '**application/json**; charset=utf-8', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"d8a79dd6eb8403cd99edd68f10fbc429697f719ddd9945d5227a3366fa41bfdc"', 'Last-Modified': 'Thu, 30 Sep 2021 23:53:20 GMT', 'X-Poll-Interval': '60', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'Content-Security-Policy': "default-src 'none'", 'Content-Encoding': 'gzip', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '59', 'X-RateLimit-Reset': '1633245644', 'X-RateLimit-Resource': 'core', 'X-RateLimit-Used': '1', 'Accept-Ranges': 'bytes', 'Content-Length': '2342', 'X-GitHub-Request-Id': 'E536:0CC3:2B2BF77:2D3E14E:61594BBC'}

Note the `Content-Type` of the response:

In [2]:
print(response.headers['Content-Type'])

application/json; charset=utf-8


The response is in JSON format, which is an open format for exchanging semi-structured data. (JSON stands for **J**ava**S**cript **O**bject **N**otation.) JSON is designed to be human-readable and machine-readable, and maps especially well in Python to nested dictionaries. Let's take a look.

> See also [this tutorial](https://www.w3schools.com/js/js_json_intro.asp) for a JSON primer. JSON is among _the_ universal formats for sharing data on the web; see, for instance, https://www.sitepoint.com/10-example-json-files/.

In [3]:
import json
print(type(response.json()))
print(json.dumps(response.json()[:3], sort_keys=True, indent=2))

<class 'list'>
[
  {
    "actor": {
      "avatar_url": "https://avatars.githubusercontent.com/u/91442574?",
      "display_login": "bxiao98",
      "gravatar_id": "",
      "id": 91442574,
      "login": "bxiao98",
      "url": "https://api.github.com/users/bxiao98"
    },
    "created_at": "2021-09-30T23:53:20Z",
    "id": "18232436745",
    "org": {
      "avatar_url": "https://avatars.githubusercontent.com/u/6626190?",
      "gravatar_id": "",
      "id": 6626190,
      "login": "hpcgarage",
      "url": "https://api.github.com/orgs/hpcgarage"
    },
    "payload": {
      "action": "started"
    },
    "public": true,
    "repo": {
      "id": 202786912,
      "name": "hpcgarage/prymer",
      "url": "https://api.github.com/repos/hpcgarage/prymer"
    },
    "type": "WatchEvent"
  },
  {
    "actor": {
      "avatar_url": "https://avatars.githubusercontent.com/u/38662177?",
      "display_login": "timothywlim",
      "gravatar_id": "",
      "id": 38662177,
      "login": "timothy

**Exercise 0.** It should be self-evident that the JSON response above consists of a sequence of records, which we will refer to as _events_. Each event is associated with an _actor_. Write some code to extract a dictionary of all actors, where the key is the actor's login and the value is the actor's URL.

In [43]:
def extract_actors (json_github_events):
    """Given JSON records for events in a GitHub repo,
    returns a dictionary of the actors and their URLs.
    """
    return {json_github_events[i]['actor']['login']: json_github_events[i]['actor']['url'] \
            for i in range(len(json_github_events))}

In [44]:
actor_urls = extract_actors(response.json ())

for actor, url in actor_urls.items ():
    print ('{}: {}'.format(actor, url))
    assert url == "https://api.github.com/users/{}".format(actor)

bxiao98: https://api.github.com/users/bxiao98
timothywlim: https://api.github.com/users/timothywlim
lexicalx: https://api.github.com/users/lexicalx
stevehamilton: https://api.github.com/users/stevehamilton
zhanglijiao0713: https://api.github.com/users/zhanglijiao0713


**Exercise 1.** Write some code that goes to each actor's URL and determines their name. If an actor URL is invalid, that actor should not appear in the output.

In [56]:
def lookup_names (actor_urls):
    """Given a dictionary of (actor, url) pairs, looks up the JSON at
    the URL and extracts the user's name (if any). Returns a new
    dictionary of (actor, name) pairs.
    """
    result = dict()
    for actor in actor_urls:
        try:
            response = requests.get(actor_urls[actor])
            result[actor] = response.json()['name']
        except:
            pass
    return result

In [57]:
actor_names = lookup_names (actor_urls)

for actor, name in actor_names.items ():
    print ("{}: {}".format (actor, name))

#assert actor_names['SamDuan'] == 'Samuel Duan'
#assert actor_names['aelipinski'] == 'OZ10'
#assert actor_names['rvuduc'] == 'Rich Vuduc (personal account)'

print("\n(Passed.)")

bxiao98: None
timothywlim: Timothy Lim
lexicalx: None
stevehamilton: Steve Hamilton
zhanglijiao0713: None

(Passed.)


That's the end of this notebook. Processing JSON is fairly straightforward, because it maps very naturally to nested dictionaries in Python. You might search the web for other sources of JSON data, including [this one](https://www.yelp.com/dataset/challenge), and do your own processing!